In [1]:
import cv2 as cv
import os
import numpy as np
from PIL import Image
import pytesseract

In [5]:
# load image
img = cv.imread('coCZPhtn8fU.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY);
 # convert to grayscale

otsu_thresh_val = cv.threshold(gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
high_thresh_val = otsu_thresh_val[0]
lower_thresh_val = otsu_thresh_val[0] * 0.2

#edged = cv.Canny(image, lower_thresh_val, high_thresh_val)
edged = cv.Canny(img, 100, 350)

image, contours, hierarchy = cv.findContours(edged, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
#contours = sorted(contours, key=cv.contourArea, reverse=True)[1:]

def crop_minAreaRect(img, rect):
    # Source: https://stackoverflow.com/questions/37177811/

    # rotate img
    angle = rect[2]
    rows,cols = img.shape[0], img.shape[1]
    matrix = cv.getRotationMatrix2D((cols/2,rows/2),angle,1)
    img_rot = cv.warpAffine(img,matrix,(cols,rows))

    # rotate bounding box
    rect0 = (rect[0], rect[1], 0.0)
    box = cv.boxPoints(rect)
    pts = np.int0(cv.transform(np.array([box]), matrix))[0]
    pts[pts < 0] = 0

    # crop and return
    return img_rot[pts[1][1]:pts[0][1], pts[1][0]:pts[2][0]]

# Find object with the biggest bounding box
mx_rect = (0,0,0,0)      # biggest skewed bounding box
mx_area = 0
for cont in contours:
    arect = cv.minAreaRect(cont)
    area = arect[1][0]*arect[1][1]
    if area > mx_area:
        mx_rect, mx_area = arect, area

# Output to files
roi = crop_minAreaRect(img, mx_rect)
cv.imwrite('Image_crop.jpg', roi)

box = cv.boxPoints(mx_rect)
box = np.int0(box)
cv.drawContours(img,[box],0,(200,0,0),2)
cv.imwrite('Image_cont.jpg', img)

pytesseract.pytesseract.tesseract_cmd = 'D:/Soft/Tesseract-OCR/tesseract.exe'
tessdata_dir_config = '--tessdata-dir "D:/Soft/Tesseract-OCR/tessdata"'
print(pytesseract.image_to_string(Image.open('Image_crop.jpg'), lang='rus', config = tessdata_dir_config))

cv.waitKey(0)
cv.destroyAllWindows()

Молоко «Коровка из
Кореновки»

    
 

56 руб.


In [87]:
#img = find_card_file('q5Y4XFYojAc.jpg')
#cv.imshow('img', img)



#img = cv.imread('q5Y4XFYojAc.jpg', 1)
#temp = cv.Canny(img, 100, 340)
#ret,thresh = cv.threshold(temp,128,255,0)
#im2, contours, hierarchy = cv.findContours(thresh,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)

#cv.imshow("canny", temp)
#cv.imshow("result_img", im2)